In [1]:
# This notebook transfers previously trained model and runs 9 more epochs. 
# It will take around 45 minutes to complete.


In [1]:
using Knet , CUDA, IterTools
using  Test, Base.Iterators, Printf, LinearAlgebra, Random
include("../tree.jl")
#include("../util.jl")
Knet.atype() = KnetArray{Float32}
using JLD2
using ArgParse
using Distributions
using Statistics
using Images

┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1278
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ImageAxes [2803e5a7-5153-5ecf-9a86-9b4c37f5f5ac]
│   exception = Required dependency ArrayInterface [4fba245c-0d91-5ea0-9b3e-6abc04ee57a9] failed to load from a cache file.
└ @ Base loading.jl:1042
┌ Warning: Module ImageAxes with build ID 1230978408942065 is missing from the cache.
│ This may mean ImageAxes [2803e5a7-5153-5ecf-9a86-9b4c37f5f5ac] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
│   exception = ErrorException("Required dependency ArrayInterface [4fba245c-0d91-5ea0-9b3e-6abc04ee57a9] failed to load from a cache file.")
└ @ Base loading.jl:1042
┌ Info: Precompiling ImageAxes [2803e5a7-5153-5ecf-9a86-9b4c37f5f5ac]
└ 

In [2]:
mutable struct SymbolsManager
    symbol2idx#::Dict{String,Int}
    idx2symbol#::Dict{Int,String}
    vocab_size#::Int
    whether_add_special_tags#::Bool
end

function SymbolsManager(whether_add_special_tags)
    symbol2idx = Dict()
    idx2symbol = Dict()
    vocab_size = 0
    symbols_manager = SymbolsManager(symbol2idx, idx2symbol, vocab_size, whether_add_special_tags)
    if whether_add_special_tags
        add_symbol(symbols_manager,"<S>")
        add_symbol(symbols_manager, "<E>")
        add_symbol(symbols_manager, "<U>")
        # non-terminal symbol = 4
        add_symbol(symbols_manager, "<N>")
        
    end
    return symbols_manager
end

function add_symbol(m::SymbolsManager, s)
    if s ∉ keys(m.symbol2idx)
        m.vocab_size = m.vocab_size + 1
        merge!(m.symbol2idx, Dict(s => m.vocab_size))
        merge!(m.idx2symbol, Dict(m.vocab_size=>s))
    end
    return m.symbol2idx[s]
end

function get_symbol_idx(s_man::SymbolsManager, s)
    if s ∉ keys(s_man.symbol2idx)
        if s_man.whether_add_special_tags
            return s_man.symbol2idx["<U>"]
        else
            println("this should never be reached (always add <U>")
            return 0
        end
    end
    return s_man.symbol2idx[s]
end

function get_idx_symbol(s_man::SymbolsManager, idx)
    if idx ∉ keys(s_man.idx2symbol)
        return "<U>"
    end
    return s_man.idx2symbol[idx]
end

function init_from_file(s_man::SymbolsManager, fn, min_freq, max_vocab_size)
    open(fn) do file
        for line in eachline(file)
            l_list = split(strip(line), "\t")
            c = parse.(Int,l_list[2] )
            if c >= min_freq
                add_symbol(s_man, l_list[1])
            end
            if s_man.vocab_size >= max_vocab_size
                break
            end
        end
    end
end

function get_symbol_idx_for_list(s_man::SymbolsManager,l)
    r = []
    for i in 1:(length(l))
        push!(r, get_symbol_idx(s_man, l[i]))
    end
    return r
end
#=
function SymbolsManager:get_symbol_idx_for_list(l)
  local r = {}
  for i = 1, #l do
    table.insert(r, self:get_symbol_idx(l[i]))
  end
  return r
end
=#

get_symbol_idx_for_list (generic function with 1 method)

In [3]:
function convert_to_tree(r_list, i_left, i_right, form_manager)
    t = Tree()
    level = 0
    left = -1
    for i in range(i_left,stop=i_right)
        if r_list[i] ==  get_symbol_idx(form_manager,"(")
            if level == 0
                left = i
            end
            level = level + 1
        elseif r_list[i] == get_symbol_idx(form_manager ,")")
            level = level - 1
            if level == 0
                c = convert_to_tree(r_list, left + 1, i - 1, form_manager)
                add_child(t, c)
            end
        elseif level == 0
            add_child(t, r_list[i])
            
        end
    end
    return t
end

convert_to_tree (generic function with 1 method)

In [4]:


mutable struct ResourceManager
    n#::Dict{String,Int}
    resource_pool#::Dict{Int,String}
    occupied_dict#::Int
    index_dict#::Bool
end

function reset(m::ResourceManager, n)
    m.n = n
    m.resource_pool = []
    for i in range(1, stop=n)
        push!(m.resource_pool, i)
    end
    m.occupied_dict = Dict()
    m.index_dict = Dict()
end

function allocate(m::ResourceManager)
    i =  length(m.resource_pool)
    deleteat!(m.resource_pool, i)
    m.occupied_dict[i] = true
    return i
end

function allocate2(m::ResourceManager, k1, k2)
    i =  allocate(m)
    if k1 ∉ keys(m.index_dict)
        m.index_dict[k1] = Dict()
    end
    m.index_dict[k1][k2] = i
    return i
end

function get2(m::ResourceManager, k1, k2)
    if ( k1 in keys(m.index_dict) )
            return nothing
    end
    if ( k1 ∉ keys(m.index_dict) )
        
        return nothing
        
    end
    i = m.index_dict[k1][k2]
    if (m.occupied_dict[i])
        return i
    else
        return nothing
    end
end

function free(m::ResourceManager, i)
    push!(m.resource_pool, i)
    m.occupied_dict[i] = nothing
end

free (generic function with 1 method)

In [22]:
function process_train_data()
    word_manager = SymbolsManager(true)

    init_from_file(word_manager, "../data/vocab.q.txt", 2, 15000)
    form_manager = SymbolsManager(true)

    init_from_file(form_manager,"../data/vocab.f.txt", 0, 15000)
    data = []
    open("../data/train.txt", "r") do file
        for line in eachline(file)
            l_list = split(line, "\t")
            w_list = get_symbol_idx_for_list(word_manager, split(strip(l_list[1]), " "))
            #w_list = KnetArray{Float32}(w_list)
            r_list = get_symbol_idx_for_list(form_manager, split(strip(l_list[2]), " "))
            #r_list = KnetArray{Float32}(r_list)
            cur_tree = convert_to_tree(r_list, 1, length(r_list), form_manager)
            push!(data, (w_list, r_list, cur_tree))
        end
    end
    out_mapfile = "../data/" * "/map.jld2"
    out_datafile = "../data/" * "/train.jld2"
    managers = [word_manager, form_manager]

    @save out_mapfile managers
    @save out_datafile  data
end




function serialize_data( name)
    data = []
    @load  ("../data/" * "/map.jld2") managers
    word_manager, form_manager = managers
    open("../data/$name.txt", "r") do file
        for line in eachline(file)
            l_list = split(line, "\t")
            w_list = get_symbol_idx_for_list(word_manager, split(strip(l_list[1]), " "))
            #w_list = KnetArray{Float32}(w_list)
            r_list = get_symbol_idx_for_list(form_manager, split(strip(l_list[2]), " "))
            #r_list = KnetArray{Float32}(r_list)
            cur_tree = convert_to_tree(r_list, 1, length(r_list), form_manager)
            push!(data, (w_list, r_list, cur_tree))
        end
    end
    out_datafile = "../data/" * "/$name.jld2"
    @save out_datafile  data
    #-------
end


Random.seed!(123)
process_train_data()
serialize_data( "test")
serialize_data( "dev")

LoadError: [91mBoundsError: attempt to access 1-element Array{Any,1} at index [3][39m

In [38]:
struct MinibatchLoader
    enc_batch_list
    enc_len_batch_list
    dec_batch_list
    num_batch
    
end

function MinibatchLoader( mode, using_gpu)
    path_data = "../data/" * "/" * string(mode) * ".jld2"
    @load  ( "../data/" * "/" * string(mode) * ".jld2") data
    
    # -- batch padding
    if length(data) % 20 != 0
        n = length(data)
        for i in 1:(length(data)%20) 
            insert!(data, n-i+1, data[n-i+1])
        end
    end
    
    enc_batch_list = []
    enc_len_batch_list = []
    dec_batch_list = []
    
    p = 0
    while p + 20 <= length(data)
        # -- build enc matrix
        max_len = length(data[p + 20 ][1])
        m_text = zeros(Int64, (20, max_len + 2))
        enc_len_list = []
        # -- add <S>
        m_text[:,1] .= 1
        for i in 1:(20 )
            w_list = data[p + i][1]
            for j in range(1, stop=length(w_list))
                # -- reversed order
                m_text[i,j+1] = w_list[length(w_list) - j + 1]
            end
            # -- add <E> (for encoder, we need dummy <E> at the end)
            for j in range(length(w_list)+2,stop=max_len+2)
                m_text[i, j] = 2
            end
            
            push!(enc_len_list, length(w_list)+2)
        end

        #enc_len_list = KnetArray{Float32}(enc_len_list)
        push!(enc_batch_list, m_text)
        push!(enc_len_batch_list, enc_len_list)
        # -- build dec tree list
        tree_batch = []
        for i in range(1, stop=20)
            push!(tree_batch, data[p+i][3])
        end
        push!(dec_batch_list, tree_batch)
        
        p = p + 20
    end
    num_batch = length(enc_batch_list)
    return MinibatchLoader(enc_batch_list, enc_len_batch_list, dec_batch_list, num_batch)
end

Base.IteratorSize(::Type{MinibatchLoader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MinibatchLoader}) = Base.HasEltype()


function Base.iterate(mbl::MinibatchLoader, state=nothing)
    if (state == nothing) & (mbl.enc_batch_list != nothing)
        next_enc_batch, state_a = iterate(mbl.enc_batch_list)
        next_enc_len_batch, state_b = iterate(mbl.enc_len_batch_list)
        next_sec_batch, state_c = iterate(mbl.dec_batch_list)
    elseif (state > length(mbl.enc_batch_list))
        return nothing
    elseif (state <= length(mbl.enc_batch_list)) 
            next_enc_batch, state_a = iterate(mbl.enc_batch_list, state)
            next_enc_len_batch, state_b = iterate(mbl.enc_len_batch_list, state)
            next_sec_batch, state_c = iterate(mbl.dec_batch_list, state)
    elseif (state == nothing) & (mbl.enc_batch_list == nothing)
        return nothing
    end
    if next_enc_batch == nothing 
        return nothing
    else
        return (next_enc_batch, next_enc_len_batch, next_sec_batch) , state_a 
    end
end
Base.IteratorSize(::Type{MinibatchLoader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MinibatchLoader}) = Base.HasEltype()

function random_batch(mbl::MinibatchLoader)
    p = rand(1:mbl.num_batch,  1)
    
    return mbl.enc_batch_list[p], mbl.enc_len_batch_list[p][1], mbl.dec_batch_list[p][1]
end

function all_batch(mbl::MinibatchLoader)
    r = []
    for p in 1:(mbl.num_batch)
        push!(r, [mbl.enc_batch_list[p], mbl.enc_len_batch_list[p], mbl.dec_batch_list[p]])
    end
    return r
end

all_batch (generic function with 1 method)

In [7]:
function norm_tree(r_list, form_manager)
    q = [convert_to_tree(r_list, 1, length(r_list), form_manager)]
    head = 1
    while head <= length(q)
        t = q[head]
        # if this level is ``and'' operator
        if (t.children[1] == get_symbol_idx(form_manager, "and")) || (t.children[1] == get_symbol_idx(form_manager, "or"))
            # sort the following subchildren
            k = Dict()
            for i in range(2, stop=length(t.children))
                if t.children[i] isa Tree
                    k[to_string(t.children[i])] = i
                else
                    k[string.(t.children[i])] = i
                end
            end
            sorted_t_dict = Dict()
            j = 2
            for keyd in sort(k; byvalue=true)
                sorted_t_dict[j] = t.children[keyd[2]]
                j=j+1
            end
            for i in range(2, stop=length(t.children))
                if i in keys(sorted_t_dict)
                    t.children[i] = sorted_t_dict[i]
                else
                    t.children[i] = []
                end
            end
        end
        # add children to q
        for i in range(1, stop=length(t.children))
            if t.children[i] isa Tree
                push!(q, t.children[i])
            end
        end
        head = head + 1
    end
    return q[1]
end

#pairsByKeys function unnecessary

function is_all_same(c1, c2)
    if length(c1) == length(c2)
        all_same = true
        for j in 1:(length(c1))
            if c1[j] != c2[j]
                all_same = false
                break
            end
        end
        return all_same
    else
        return false
    end
end

function compute_accuracy(candidate_list, reference_list)
    if length(candidate_list) != length(reference_list)
        #print("candidate list has length {}, reference list has length {}\n".format(len(candidate_list), len(reference_list)))
    end
    len_min = min(length(candidate_list), length(reference_list))
    c = 0
    for i in 1:(len_min)
        if is_all_same(candidate_list[i], reference_list[i])
            c = c+1
        end
    end
    return c/float.(len_min)
end

function compute_tree_accuracy(candidate_list_, reference_list_, form_manager)
    candidate_list = []
    for i in 1:(length(candidate_list_))
        push!(candidate_list, to_list(norm_tree(candidate_list_[i], form_manager), form_manager))  
    end
    reference_list = []
    for i in 1:(length(reference_list_))
        push!(reference_list, to_list(norm_tree(reference_list_[i], form_manager), form_manager))
    end
    return compute_accuracy(candidate_list, reference_list)
end

compute_tree_accuracy (generic function with 1 method)

In [8]:

struct Attention; end

struct Embed; w; end

struct Linear; w; b; end

struct Memory; w; end

struct L2L
    enc_lstm
    dec_lstm
    attention
#    memory
    srcembed
    tgtembed
    dropout
    projection_1
    projection_2
    opt
    word_manager
    form_manager
end

In [9]:

function uniform(Flo::Type{Float32}, dim)
    return rand(MersenneTwister(123),Flo,dim)  * (0.08 + 0.08) .- 0.08
end

function uniform_attn(Flo::Type{Float32}, dim)
    return rand(Flo,dim)  * (0.08 + 0.08) .- 0.08
end

# You can use the following helper function for scaling and linear transformations of 3-D tensors:
mmul(w,x) = (w == 1 ? x : w == 0 ? 0 : reshape(w * reshape(x,size(x,1),:), (:, size(x)[2:end]...)))

mmul (generic function with 1 method)

In [10]:
function Embed( opt, vocabsize::Int, embedsize::Int)
    return Embed(param(uniform(Float32, (embedsize,vocabsize,1))))
end

function Linear(opt, inputsize::Int, outputsize::Int)
    return Linear(param(uniform(Float32, (outputsize,inputsize))),param0(outputsize))
end

function Linear_attn(opt, inputsize::Int, outputsize::Int)
    return Linear_attn(param(uniform_attn(Float32, (outputsize,inputsize))),param0(outputsize))
end


Linear_attn (generic function with 1 method)

In [11]:
function (l::Embed)(x)

    x_copy = copy(x)
    for i in 1:size(x_copy, 1)
        if x_copy[i] == 0
            x_copy[i] = 2
        end
    end

    return l.w[:,x_copy]

    #return l.w[:,x]
end

function (l::Linear)(x)
    return (mmul(l.w , x) .+ l.b)
end

In [12]:

struct Embed2; w; end
function Embed2(opt, vocabsize::Int, embedsize::Int)
    return Embed2(param(uniform(Float32, (embedsize,vocabsize+1,1))))
end

function (l::Embed2)(x)

    x_copy = copy(x)
    for i in 1:size(x_copy, 1)
        if x_copy[i] == 0
            x_copy[i] = 171
        end
    end

    return l.w[:,x_copy]
end

In [59]:
function L2L(src_input_size, tgt_input_size, opt, form_manager, word_manager)
    srcembed = Embed(opt, src_input_size  , opt["rnn_size"])
    tgtembed = Embed2(opt, tgt_input_size , opt["rnn_size"])
    enc_lstm = RNN(opt["rnn_size"], opt["rnn_size"]; winit=uniform, binit=uniform, finit=uniform, dropout=0.1,  rnnType=:lstm)
    dec_lstm = RNN(2*opt["rnn_size"], opt["rnn_size"];   winit=uniform, binit=uniform, finit=uniform, dropout=0.1, rnnType=:lstm)
    attention = Attention( )
#    memory = Memory(param(uniform(Float32, (opt["rnn_size"],opt["rnn_size"]))))
    dropout = opt["dropoutrec"]
    projection_1 = Linear(opt, 2*opt["rnn_size"],  opt["rnn_size"] )
    projection_2 = Linear(opt, opt["rnn_size"],  tgt_input_size  )
    opt = opt
    form_manager = form_manager
    word_manager = word_manager
    return L2L(enc_lstm, dec_lstm,  attention, 
       # memory, 
        srcembed, tgtembed, dropout, projection_1, projection_2, opt, word_manager,form_manager)
end


function encode(l::L2L, src)
    
    src_embedding = l.srcembed(src)
    encoded = l.enc_lstm(src_embedding)
    encoded_res = src_embedding + encoded
    return  encoded_res
end


#function (m::Memory)(x)
#    keys = mmul(m.w, permutedims(x, (2,1,3)))
#    return permutedims(keys,(2,1,3)), x
#end

function (a::Attention)(src_encoded, decoder_output)
    attention_scores = bmm(   src_encoded, permutedims(decoder_output, (1,3,2)))
    attention = Knet.softmax(attention_scores, dims=1)
    context_vector = bmm(permutedims(attention, (2,1,3)), src_encoded)
    return vcat(permutedims(context_vector, (2,3,1)), decoder_output ) 
end

function attention_decode(l::L2L, src_encoded, decoder_output)

    attention_vector = l.attention(src_encoded, decoder_output)
    projected_mid  = tanh.(l.projection_1(attention_vector))
    residual1 = projected_mid + decoder_output
    dropped_mid = dropout(residual1, 0.3)
    return l.projection_2(dropped_mid)
end


#function (a::Attention)(src_encoded, decoder_output)
#    keys, src_encoded = src_encoded
#    attention_scores = bmm( keys , permutedims(decoder_output, (1,3,2)))
#    attention = Knet.softmax(attention_scores; dims=1)
#    context_vector = bmm(permutedims(attention, (2,1,3)), src_encoded)
#    return vcat(permutedims(context_vector, (2,3,1)), decoder_output )
#end
     

function decode(l::L2L, tgt, prev_c, prev_h, parent_h)

    input_tgt = l.tgtembed(tgt)
    input = cat(input_tgt, parent_h, dims=1)
    l.dec_lstm.h, l.dec_lstm.c = prev_h, prev_c
    decoder_output = l.dec_lstm(input)
    return   decoder_output
end


#function attention_decode(l::L2L, src_encoded, decoder_output)
#    keys, src_encoded  = l.memory(src_encoded)
#    attention_vector = l.attention((keys, src_encoded), decoder_output)
#    projected_mid  = tanh.(l.projection_1(attention_vector))
#    dropped_mid = dropout(projected_mid, 0.3)
#    return l.projection_2(dropped_mid)
#end

function (l::L2L)(enc_batch, enc_len_batch, dec_tree_batch )
    enc_max_len =  size(enc_batch,2)
    dec_s = Dict()
    enc_s = Dict()
    
    
    for i in range(0,stop=l.opt["enc_seq_length"] )
        enc_s[i] = Dict()
        for j in range(1,stop=2)
            enc_s[i][j] = KnetArray(zeros(Float32, l.opt["rnn_size"],l.opt["batch_size"],1))
        end
    end
    
    for i in range(0,stop=l.opt["dec_seq_length"] )
        dec_s[i] = Dict()
        for j in range(0,stop=l.opt["dec_seq_length"])
            dec_s[i][j] = Dict()
            for k in range(1,stop=2)
                dec_s[i][j][k] = KnetArray(zeros(Float32, l.opt["rnn_size"],l.opt["batch_size"],1))
            end
        end
    end
    #dec_rnns_manager = ResourceManager(0,0,0,0)
    #reset(dec_rnns_manager, l.opt["dec_seq_length"])
    
    #criterions=[]
    #for i in range( 1, stop=l.opt["dec_seq_length"])
    #    push!(criterions, Knet.nll)
    #end

    l.enc_lstm.c , l.enc_lstm.h = 0, 0
    for i in range(1, stop=enc_max_len)
        encdd = encode(l, enc_batch[:,i])
        enc_s[i][1] =  l.enc_lstm.h
        enc_s[i][2] =  l.enc_lstm.c
    end
    
    enc_s_top = 0
    for i in range(1,stop=enc_max_len)
        if i == 1
            enc_s_top = enc_s[i][1]
        else
            enc_s_top = cat(enc_s_top,enc_s[i][1], dims=3)
        end
    end
    enc_s_top = permutedims(enc_s_top, (3,1,2))

    
    # tree decode
    queue_tree = Dict()
    for i in range(1, stop=l.opt["batch_size"])
        queue_tree[i] = []
        push!(queue_tree[i],Dict("tree"=>dec_tree_batch[i], "parent"=>0, "child_index"=>1))
    end
    loss = 0
    cur_index, max_index = 1,1
    dec_batch = Dict()
    softmax_predictions = Dict()
    while (cur_index <= max_index)
        # build dec_batch for cur_index
        max_w_len = -1
        batch_w_list = []
        for i in range(1, stop=l.opt["batch_size"])
            w_list = []
            if (cur_index <= length(queue_tree[i]))
                t = queue_tree[i][cur_index ]["tree"]
                for ic in range(1, stop=t.num_children)
                    if t.children[ic] isa Tree
                        push!(w_list, 4)
                        push!(queue_tree[i], Dict("tree"=>t.children[ic], "parent"=>cur_index, "child_index"=>ic))
                    else
                        push!(w_list, t.children[ic])
                    end
                end
                if length(queue_tree[i]) > max_index
                    max_index = length(queue_tree[i])
                end
            end
            if length(w_list) > max_w_len
                max_w_len = length(w_list)
            end
            push!(batch_w_list, w_list)
        end
        dec_batch[cur_index] = zeros(Int64, l.opt["batch_size"], max_w_len + 2)


        for i in range(1, stop=l.opt["batch_size"])
            w_list = batch_w_list[i]
            if length(w_list) > 0
                for j in range(1, stop=length(w_list))
                    dec_batch[cur_index][i,j+1] = w_list[j]
                end
                # add <S>, <E>
                if cur_index == 1
                    dec_batch[cur_index][i,1] = 1
                else
                    dec_batch[cur_index][i,1] = get_symbol_idx(l.form_manager, "(")
                end    
                dec_batch[cur_index][i,length(w_list) + 2] = 2
            end
        end


        
        if cur_index == 1
            for i in range(1, stop=l.opt["batch_size"])
                if i == 1
                    dec_s[1][0][1] = enc_s[enc_len_batch[i]][1][:,i,:]
                    dec_s[1][0][2] = enc_s[enc_len_batch[i]][2][:,i,:]
                else
                    dec_s[1][0][1] = cat(dec_s[1][0][1], enc_s[enc_len_batch[i]][1][:,i,:], dims=2)
                    dec_s[1][0][2] = cat(dec_s[1][0][2], enc_s[enc_len_batch[i]][2][:,i,:], dims=2)
                end
            end
        else
            for i in range(1, stop=l.opt["batch_size"])
                if (cur_index <= length(queue_tree[i]))
                    par_index = queue_tree[i][cur_index ]["parent"]
                    child_index = queue_tree[i][cur_index ]["child_index"]
                    if i == 1
                        dec_s[cur_index][0][1] = dec_s[par_index][child_index][1][:,i,:]
                        dec_s[cur_index][0][2] = dec_s[par_index][child_index][2][:,i,:]
                    else
                        dec_s[cur_index][0][1] = cat(dec_s[cur_index][0][1],dec_s[par_index][child_index][1][:,i,:],dims=2)
                        dec_s[cur_index][0][2] = cat(dec_s[cur_index][0][2],dec_s[par_index][child_index][2][:,i,:],dims=2)
                    end
                else
                    if i==1
                        dec_s[cur_index][0][1] = KnetArray(zeros(Float32, l.opt["rnn_size"],1))
                        dec_s[cur_index][0][2] = KnetArray(zeros(Float32, l.opt["rnn_size"],1))
                    else
                        dec_s[cur_index][0][1] = cat(dec_s[cur_index][0][1],KnetArray(zeros(Float32, l.opt["rnn_size"],1)),dims=2)
                        dec_s[cur_index][0][2] = cat(dec_s[cur_index][0][2],KnetArray(zeros(Float32, l.opt["rnn_size"],1)),dims=2)
                    end
                end
            end
        end
        softmax_predictions[cur_index] = Dict()
        parent_h = dec_s[cur_index][0][1]

        for i in range(1, stop=size(dec_batch[cur_index], 2) - 1)
            #i_dec_rnns = allocate2(dec_rnns_manager, cur_index, i)
            decd = decode(l, dec_batch[cur_index][:,i,:], dec_s[cur_index][i - 1][2], dec_s[cur_index][i - 1][1],  parent_h)

            dec_s[cur_index][i][1], dec_s[cur_index][i][2] = l.dec_lstm.h, l.dec_lstm.c

            softmax_predictions[cur_index][i] = attention_decode(l, enc_s_top, dec_s[cur_index][i][1])
            loss = loss + Knet.nll(softmax_predictions[cur_index][i], dec_batch[cur_index][:,i+1,:]; dims=1, average=true)
            #loss = loss + criterions[i_dec_rnns](softmax_predictions[cur_index][i], dec_batch[cur_index][:,i+1,:]; dims=1, average=true)
        end
        
        cur_index = cur_index + 1
    end

    return loss # / l.opt["batch_size"]
end


In [14]:

opt = Dict("data_dir" => "../data/", "checkpoint_dir" => "checkpoint_dir", "rnn_size" => 200, "dropout" => 0.3,
"dropoutrec" => 0.3, "enc_seq_length" => 60, "dec_seq_length" => 220, "batch_size"=>20, "max_epochs"=>130,
"learning_rate" => 0.007, "init_weight"=> 0.08, "learning_rate_decay"=> 0.98, "learning_rate_decay_after"=>5,
"decay_rate"=>0.95, "grad_clip"=>5)

Dict{String,Any} with 15 entries:
  "rnn_size"                  => 200
  "enc_seq_length"            => 60
  "batch_size"                => 20
  "data_dir"                  => "../data/"
  "checkpoint_dir"            => "checkpoint_dir"
  "learning_rate_decay"       => 0.98
  "dropout"                   => 0.3
  "init_weight"               => 0.08
  "max_epochs"                => 130
  "grad_clip"                 => 5
  "learning_rate"             => 0.007
  "decay_rate"                => 0.95
  "dropoutrec"                => 0.3
  "dec_seq_length"            => 220
  "learning_rate_decay_after" => 5

In [15]:



function convert_to_string(idx_list, form_manager)
    w_list = []
    for i in range(1, stop=length(idx_list))
        push!(w_list, get_idx_symbol(form_manager, Int(idx_list[i])))
    end
    return join(w_list, " ")
end

function do_generate(l::L2L,  enc_w_list, word_manager, form_manager)
    enc_w_list_withSE = value(enc_w_list)
    push!(enc_w_list_withSE, get_symbol_idx(word_manager,"<S>"))
    insert!(enc_w_list_withSE, 1, get_symbol_idx(word_manager, "<E>"))
    enc_s_top = KnetArray(zeros(Float32, opt["rnn_size"], 1, length(enc_w_list_withSE)))
    l.enc_lstm.c , l.enc_lstm.h = 0, 0
    for i in range(length(enc_w_list_withSE), 1, step=-1)
        enc_res = encode(l,  enc_w_list_withSE[i])
        enc_s_top[:,:,length(enc_w_list_withSE)-i+1] = l.enc_lstm.h
    end
    
    # decode
    s = (l.enc_lstm.h , l.enc_lstm.c)
    queue_decode = []
    push!(queue_decode, Dict("s"=> s, "parent"=>0, "child_index"=>1, "t"=> Tree()))
    head = 1
    enc_s_top = permutedims(enc_s_top, (3,1,2))
    while (head <= length(queue_decode)) & ( head <= 100)
        s = queue_decode[head]["s"]
        parent_h = s[1]
        t = queue_decode[head]["t"]
        if head == 1
            prev_word = get_symbol_idx(form_manager, "<S>")
            
        else
            prev_word = get_symbol_idx(form_manager, "(")
        end
        i_child = 1
        
        while true
            dec_res = decode(l, prev_word, s[2],s[1], parent_h)
            s = (l.dec_lstm.h, l.dec_lstm.c)
            prediction = attention_decode(l, enc_s_top, s[1])
            pred = argmax(prediction, dims=1)
            prev_word = pred[1][1]
            
            if (prev_word == get_symbol_idx(form_manager,"<E>")) || (t.num_children >= opt["dec_seq_length"]) 
                break
            elseif (prev_word == get_symbol_idx(form_manager, "<N>")) 
                push!(queue_decode, Dict("s"=>s, "parent"=>head, "child_index"=>i_child, "t"=>Tree()))
                add_child(t,prev_word)
            else
                add_child(t,prev_word)
            end
            i_child = i_child + 1
        end
        head = head + 1
    end
    # refine the root tree
    for i in range(length(queue_decode), 2, step=-1)
        cur = queue_decode[i]
        queue_decode[cur["parent"]]["t"].children[cur["child_index"]] = cur["t"]
    end

    return to_list(queue_decode[1]["t"], form_manager)
end


do_generate (generic function with 1 method)

In [34]:
Random.seed!(1234)
reference_list = []
candidate_list = []
@load  (string(opt["data_dir"]) * "/test.jld2") data

m_model = Knet.load("attn-1807547143885416.jld2")

m_model=m_model["model"]
@load  (string(opt["data_dir"]) * "/map.jld2") managers
word_manager, form_manager = managers
open("../output/seq2seq_output.txt", "w") do output
    for i in range(1, stop=length(data))
        x = data[i]
        reference = x[2]
        candidate = do_generate(m_model, x[1], word_manager, form_manager)
        candidate = [Int(c) for c in candidate]
        ref_str = convert_to_string(reference, form_manager)
        cand_str = convert_to_string(candidate, form_manager)
        push!(reference_list, reference)
        push!(candidate_list, candidate)
        write(output, string(ref_str) *"\n")                    
        write(output, string(cand_str) *"\n") 
    end
    val_acc = compute_tree_accuracy(candidate_list, reference_list, form_manager)
    print("ACCURACY = " * string(val_acc) * "\n")
    write(output,"ACCURACY = " * string(val_acc) *"\n") 
end

(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)
(1, 200, 1)
(4, 1, 1)


(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(6, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1

(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)
(1, 200, 1)
(7, 1, 1)


(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)


(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)
(1, 200, 1)
(8, 1, 1)


(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)


(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)
(1, 200, 1)
(9, 1, 1)


LoadError: [91mInterruptException:[39m

In [210]:

    Random.seed!(123)
    @load  (string(opt["data_dir"]) * "/map.jld2") managers
    word_manager, form_manager = managers
    train_loader = MinibatchLoader( "train", true)
   

    ##-- model 
    if !isdir(opt["checkpoint_dir"])
        mkdir(opt["checkpoint_dir"])
    end
    model = L2L(word_manager.vocab_size, form_manager.vocab_size, opt, form_manager, word_manager)

    ##-- start training -- rmsprop  adam
    step = 0
    epoch = 0
    optim_state = Dict("learningRate" => opt["learning_rate"], "alpha" =>  opt["decay_rate"])

    ctrain_loader = collect(train_loader)
    for epoch in 1:9

        progress!(rmsprop(m_model, ctrain_loader; lr=optim_state["learningRate"],rho=0.95,  gclip=5), steps=1)
        if epoch >= 0 # opt["learning_rate_decay_after"] 
            for paramtr in Knet.params(m_model); paramtr.opt.lr = paramtr.opt.lr*opt["learning_rate_decay"]; end
            optim_state["learningRate"]=optim_state["learningRate"]*opt["learning_rate_decay"]
        end
    
    end
    checkpoint = Dict()
    checkpoint["lang2logic_model"] = m_model
    checkpoint["opt"] = opt
    checkpoint["epoch"] = epoch
    Knet.save(opt["checkpoint_dir"] *"/model_seq2seq2.jld2", "checkpoint", checkpoint)

    Knet.save("attn-$(Int(time_ns())).jld2", "model", m_model)

(Any[52, 18, 59, 76, 9], Any[13, 7, 14, 5, 8, 5, 16, 7, 6, 5, 20, 7, 21, 6, 6], Tree(nothing, 4, Any[13, 7, 14, Tree(Tree(#= circular reference @-3 =#), 3, Any[8, Tree(Tree(#= circular reference @-3 =#), 2, Any[16, 7]), Tree(Tree(#= circular reference @-3 =#), 3, Any[20, 7, 21])])]))


LoadError: [91mOut of GPU memory trying to allocate 1.227 MiB[39m
[91mEffective GPU memory usage: 99.99% (31.746 GiB/31.749 GiB)[39m
[91mCUDA allocator usage: 19.328 GiB[39m
[91mbinned usage: 19.328 GiB (19.328 GiB allocated, 0 bytes cached)[39m
[91mDiscrepancy of 16.000 KiB between memory pool and allocator![39m


In [30]:
open("../output/seq2seq_output.txt", "w") do output
    for i in range(1, stop=length(data))
        x = data[i]
        reference = x[2]
        candidate = do_generate(m_model, x[1], word_manager, form_manager)
        candidate = [Int(c) for c in candidate]
        ref_str = convert_to_string(reference, form_manager)
        cand_str = convert_to_string(candidate, form_manager)
        push!(reference_list, reference)
        push!(candidate_list, candidate)
        write(output, string(ref_str) *"\n")                    
        write(output, string(cand_str) *"\n") 
    end
    val_acc = compute_tree_accuracy(candidate_list, reference_list, form_manager)
    print("ACCURACY = " * string(val_acc) * "\n")
    write(output,"ACCURACY = " * string(val_acc) *"\n") 
end
# +2

ACCURACY = 0.8113839285714286


30